# API 접속

In [1]:
from pykiwoom.kiwoom import *
import time
import pandas as pd

In [2]:
#로그인
kiwoom = Kiwoom()
print("로그인중")
kiwoom.CommConnect(block=True)
print(("로그인 완료"))

로그인중
로그인 완료


In [3]:
#연결
state = kiwoom.GetConnectState()
if state == 0:
    print('연결되지않음')
elif state == 1:
    print("연결됨")

연결됨


In [4]:
#로그인정보
account_num = kiwoom.GetLoginInfo("ACCOUNT_CNT")        # 전체 계좌수
accounts = kiwoom.GetLoginInfo("ACCNO")                 # 전체 계좌 리스트
user_id = kiwoom.GetLoginInfo("USER_ID")                # 사용자 ID
user_name = kiwoom.GetLoginInfo("USER_NAME")            # 사용자명
keyboard = kiwoom.GetLoginInfo("KEY_BSECGB")            # 키보드보안 해지여부
firewall = kiwoom.GetLoginInfo("FIREW_SECGB")           # 방화벽 설정 여부
# print(account_num)
# print(accounts)
# print(user_id)
# print(user_name)
# print(keyboard)
# print(firewall)

# 일봉데이터 수집

In [5]:
df_multi = kiwoom.block_request('opt10081',
                                 종목코드 = '035720',
                                 기준일자 = '20220706',
                                 수정주가구분 = 1,
                                 output = '주식일봉차트조회',
                                 next=0
                                 )

CommRqData opt10081 opt10081 0 0101


# 필요한 데이터만 가져오기

In [6]:
df = pd.DataFrame()
df['일자'] = pd.to_datetime(df_multi['일자'])
df['종가'] = df_multi['현재가'].astype(int)
df['거래량'] = df_multi['거래량'].astype(int)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      600 non-null    datetime64[ns]
 1   종가      600 non-null    int32         
 2   거래량     600 non-null    int32         
dtypes: datetime64[ns](1), int32(2)
memory usage: 9.4 KB


# 등락률 계산

In [8]:
#등락률 계산 : 일 등락률 = (오늘종가 – 어제종가) / 어제종가 * 100 => 평균
등락률 = []
for i in range(len(df['종가'])):
    if i == len(df['종가'])-1:
        break
    else:    
        등락률.append((df['종가'][i]-df['종가'][i+1])/df['종가'][i+1]*100)
등락률.append(0)

In [9]:
df['등락률']=등락률

# 거래변동량 계산

In [10]:
#거래변동량 계산 : 오늘거래량 - 어제거래량
거래변동량 = []
for i in range(len(df['거래량'])):
    if i == len(df['거래량'])-1:
        break
    else:    
        거래변동량.append(abs((df['거래량'][i]-df['거래량'][i+1])))
거래변동량.append(0)

In [11]:
df['거래변동량']=거래변동량

# 종가 data 정규화

In [12]:
#종가 data 정규화 MinMaxScaler()
from sklearn.preprocessing import MinMaxScaler

In [13]:
df_종가 = pd.DataFrame()
df_종가['종가'] = df['종가']

In [14]:
scaler = MinMaxScaler()
scaler.fit(df_종가)
df_scaled = scaler.transform(df_종가)

In [15]:
df_종가_scaled = pd.DataFrame(data = df_scaled)

In [16]:
df_종가_scaled.min()

0    0.0
dtype: float64

In [17]:
df_종가_scaled.max()

0    1.0
dtype: float64

In [18]:
df['정규화_종가']=df_종가_scaled
df

,일자,종가,거래량,등락률,거래변동량,정규화_종가
0,2022-07-06,73500,2724220,2.083333,598532,0.326817
1,2022-07-05,72000,2125688,5.726872,1106517,0.316298
2,2022-07-04,68100,1019171,1.188707,798105,0.288950
3,2022-07-01,67300,1817276,-3.719599,478245,0.283340
4,2022-06-30,69900,1339031,-1.271186,153560,0.301572
...,...,...,...,...,...,...
595,2020-02-10,33918,1869502,0.895380,731893,0.049255
596,2020-02-07,33617,2601395,0.903470,149198,0.047144
597,2020-02-06,33316,2750593,1.840191,473792,0.045033
598,2020-02-05,32714,2276801,-0.911707,309820,0.040812
